In [1]:
pip install pandas psycopg2-binary

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 9.1 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import psycopg2 as pg

In [3]:
import pandas as pd
import psycopg2 as pg

# Conexão com o banco de dados
engine = pg.connect("dbname='clever' user='clever' host='postgres_clever' port='5432' password='clever'")

# Consulta SQL para obter a lista de tabelas
query = """
SELECT table_schema, table_name
FROM information_schema.tables
WHERE table_type = 'BASE TABLE' AND table_schema NOT IN ('pg_catalog', 'information_schema')
ORDER BY table_schema, table_name;
"""

# Executa a consulta e armazena o resultado em um DataFrame
tables_df = pd.read_sql(query, engine)

# Exibe o DataFrame com a lista de tabelas
print(tables_df)


  table_schema                                  table_name
0       public    curated_dim_company_profiles_google_maps
1       public        curated_fact_customer_reviews_google
2       public  curated_score_company_profiles_google_maps
3       public       curated_score_customer_reviews_google
4       public        trusted_company_profiles_google_maps
5       public        trusted_fact_customer_reviews_google


/tmp/ipykernel_810/451873851.py:16: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  tables_df = pd.read_sql(query, engine)


In [4]:
query = """
SELECT 
    distinct type 
FROM 
    curated_dim_company_profiles_google_maps dc
INNER JOIN
    curated_fact_customer_reviews_google fc
ON
    dc.google_id = fc.google_id

"""

In [5]:
pd.read_sql(query, engine).to_dict()

/tmp/ipykernel_810/1327511401.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql(query, engine).to_dict()


{'type': {0: 'Real estate agency',
  1: 'Property management company',
  2: 'Real estate consultant',
  3: 'Moving and storage service',
  4: 'Real estate agent',
  5: 'Property investment',
  6: 'Mover'}}

In [8]:
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderServiceError, GeocoderTimedOut, GeocoderUnavailable

def get_coordinates(address_or_postal_code):
    """
    Receives an address or postal code and returns the latitude and longitude.
    
    Parameters:
        address_or_postal_code (str): The full address or postal code in the US.
    
    Returns:
        tuple: (latitude, longitude) if found.
        None: If unable to find coordinates or if an error occurs.
    """
    # Initialize the geocoder with a unique user_agent
    geolocator = Nominatim(user_agent="your_app_name")  # Replace "your_app_name" with the name of your application

    try:
        # Attempt to geocode the address or postal code
        location = geolocator.geocode(address_or_postal_code, timeout=10)
        
        if location:
            return (location.latitude, location.longitude)
        else:
            print("No coordinates found for the provided address or postal code.")
            return None

    except GeocoderTimedOut:
        print("Connection to the geocoding service timed out.")
        return None
    except GeocoderServiceError as e:
        print(f"Geocoding service error: {e}")
        return None
    except GeocoderUnavailable:
        print("Geocoding service is currently unavailable.")
        return None
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return None

import numpy as np
import pandas as pd

def haversine_distance(lat1, lon1, lat2, lon2):
    """
    Calculates the Haversine distance between two geographical coordinates.
    lat1, lon1 are the latitude and longitude of the first point (in radians).
    lat2, lon2 are the latitude and longitude of the second point (in radians).
    Returns the distance in kilometers.
    """
    R = 6371  # Radius of the Earth in kilometers

    # Convert degrees to radians
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])

    # Differences between the coordinates
    dlat = lat2 - lat1
    dlon = lon2 - lon1

    # Haversine formula
    a = np.sin(dlat / 2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2.0)**2
    c = 2 * np.arcsin(np.sqrt(a))
    
    return R * c

def calculate_distances(df, coord_tuple):
    """
    Receives a DataFrame with 'latitude' and 'longitude' columns and a tuple (latitude, longitude).
    Returns a new DataFrame with a 'distance' column that contains the distance from the tuple to each point.
    """
    lat1, lon1 = coord_tuple
    
    # Calculate the distance for each point in the DataFrame
    df['distance'] = df.apply(lambda row: haversine_distance(lat1, lon1, row['latitude'], row['longitude']), axis=1)
    
    return df



def get_user_input(text, options=None):
    """
    Displays a message to the user and optionally presents a list of options to choose from.

    Parameters:
    - text (str or list): The message to display. If it's a list, each item is printed on a new line.
    - options (list, optional): A list of options to present to the user. If provided, the user
                                is prompted to select one by entering the corresponding number.

    Returns:
    - The user's input. If options are provided, returns the selected option.
      Otherwise, returns the raw input string.
    """
    # Display the text
    if isinstance(text, list):
        for line in text:
            print(line)
    else:
        print(text)
    
    # If options are provided, display them and prompt for selection
    if options:
        for idx, option in enumerate(options, start=1):
            print(f"{idx}. {option}")
        
        while True:
            try:
                choice = int(input("Please choose an option by entering the corresponding number: "))
                if 1 <= choice <= len(options):
                    return options[choice - 1]
                else:
                    print(f"Please enter a number between 1 and {len(options)}.")
            except ValueError:
                print("Invalid input. Please enter a valid number.")
    else:
        # If no options are provided, simply return the user's input
        return input("Your input: ")

In [9]:
possible_actions = ["buy", "sell", "move"]

In [10]:
types_by_action = {
    "buy": [
        'Real estate agency',
        'Real estate consultant',
        'Real estate agent',
        'Property investment'
    ],
    "sell": [
        'Property management company',
        'Real estate agency',
        'Real estate consultant',
        'Real estate agent'
    ],
    "move": [
        'Moving and storage service',
        'Mover'
    ]
}

In [11]:
dict_user_options = {}

In [12]:
# 1. Coletar o nome do usuário
dict_user_options['user_name'] = get_user_input("What's your name?")

# 2. Perguntar sobre a ação desejada com opções
action_prompt = "Are you looking to buy a house, sell a house, or move?"
action_options = possible_actions
dict_user_options["action"] = get_user_input(action_prompt, options=action_options)
dict_user_options["action_types"] = types_by_action.get(dict_user_options["action"])

# 3. Coletar o endereço do usuário
dict_user_options["address"] = get_user_input("What is your address? You can also send me just your postal code.")
dict_user_options["coordinates"] = get_coordinates(dict_user_options["address"])

What's your name?


Your input:  Leonardo Mattos


Are you looking to buy a house, sell a house, or move?
1. buy
2. sell
3. move


Please choose an option by entering the corresponding number:  1


What is your address? You can also send me just your postal code.


Your input:  Miami


In [13]:
dict_user_options

{'user_name': 'Leonardo Mattos',
 'action': 'buy',
 'action_types': ['Real estate agency',
  'Real estate consultant',
  'Real estate agent',
  'Property investment'],
 'address': 'Miami',
 'coordinates': (25.7741728, -80.19362)}

In [15]:
type_items = ', '.join(f"'{t}'" for t in dict_user_options.get("action_types"))

type_filter = f"type in ({type_items})"


In [17]:

query = f"""
WITH reviews_score as (
    SELECT google_id, AVERAGE('user_review_score')
    FROM
        curated_fact_customer_reviews_google
    GROUP BY
        google_id
)

SELECT 
    dc.*, fc.company_profile_score as score
FROM 
    curated_dim_company_profiles_google_maps dc
INNER JOIN
    curated_score_company_profiles_google_maps fc
ON
    dc.google_id = fc.google_id
LEFT JOIN
    reviews_scorea
WHERE 1=1
    AND {type_filter}
ORDER BY company_profile_score DESC

"""

In [18]:
query

"\nWITH reviews_score as (\n    SELECT google_id, AVERAGE('user_review_score')\n    FROM\n        curated_fact_customer_reviews_google\n    GROUP BY\n        google_id\n)\n\nSELECT \n    dc.*, fc.company_profile_score as score\nFROM \n    curated_dim_company_profiles_google_maps dc\nINNER JOIN\n    curated_score_company_profiles_google_maps fc\nON\n    dc.google_id = fc.google_id\nLEFT JOIN\n    reviews_scorea\nWHERE 1=1\n    AND type in ('Real estate agency', 'Real estate consultant', 'Real estate agent', 'Property investment')\nORDER BY company_profile_score DESC\n\n"

In [104]:
df = pd.read_sql(query, engine)

/tmp/ipykernel_280/1589124803.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, engine)


In [105]:
df.columns

Index(['google_id', 'name', 'site', 'subtypes', 'type', 'category', 'phone',
       'full_address', 'borough', 'street', 'city', 'postal_code', 'state',
       'us_state', 'country', 'country_code', 'latitude', 'longitude',
       'time_zone', 'area_service', 'reviews_tags', 'photo', 'photos_count',
       'street_view', 'located_in', 'working_hours', 'other_hours',
       'business_status', 'about', 'posts', 'logo', 'description', 'verified',
       'owner_id', 'owner_title', 'owner_link', 'location_link', 'place_id',
       'cid', 'reviews', 'rating', 'saturday_working_hours',
       'sunday_working_hours', 'weekdays_working_hours', 'score'],
      dtype='object')

In [107]:
calculate_distances(df, dict_user_options.get("coordinates")).sort_values(["distance", "score"]).head(10)

,google_id,name,site,subtypes,type,category,phone,full_address,borough,street,...,location_link,place_id,cid,reviews,rating,saturday_working_hours,sunday_working_hours,weekdays_working_hours,score,distance
149,0x88d9b723b392a59f:0xb67456e41215189e,Local Realty Services,http://www.localrealtyservices.com/,Real estate agency,Real estate agency,Real estate agency,+1 305-216-3521,"333 NW 4th Ave, Miami, FL 33128",Downtown Miami,333 NW 4th Ave,...,https://www.google.com/maps/place/Local+Realty...,ChIJn6WSsyO32YgRnhgVEuRWdLY,13147200000000000000,20,5.0,Open 24 hours,Open 24 hours,Open 24 hours,0.770909,1.265140
1350,0x88d9b7a441034edf:0x77e07bd6736537fc,Miami Real Estate Broker,http://www.miamirealestatebroker.com/,Real estate agency,Real estate agency,Real estate agency,+1 305-469-8090,"1861 NW S River Dr, Miami, FL 33125",Grapeland Heights,1861 NW S River Dr,...,https://www.google.com/maps/place/Miami+Real+E...,ChIJ304DQaS32YgR_Ddlc9Z74Hc,8638040000000000000,5,5.0,Open 24 hours,Open 24 hours,Open 24 hours,0.678464,1.389771
528,0x88d9b7f8f50d5605:0x369392715afc7d74,Properties Miami,https://propertiesmiami.com/,"Real estate consultant, Property management co...",Real estate consultant,Real estate agency,+1 786-505-4939,"637 SW 3rd St APT 306, Miami, FL 33130",Little Havana,637 SW 3rd St APT 306,...,https://www.google.com/maps/place/Properties+M...,ChIJBVYN9fi32YgRdH38WnGSkzY,3932650000000000000,39,4.7,8 AM-10 PM,8 AM-10 PM,8 AM-10 PM,0.726645,1.427465
2613,0x88d9b741f1ba9f11:0xbbb2f9585af84c48,Gridline Properties,http://www.gridlineproperties.com/,Real estate agency,Real estate agency,Real estate agency,+1 305-507-7098,"101 NW 8th St #200, Miami, FL 33136",Overtown,101 NW 8th St #200,...,https://www.google.com/maps/place/Gridline+Pro...,ChIJEZ-68UG32YgRSEz4Wlj5srs,13525100000000000000,4,3.0,Closed,Closed,9AM-6PM,0.380497,1.552509
2307,0x88d9b6ea10c877a7:0xb4a9aa8a32d2a47b,Fausto Commercial,http://www.faustocommercial.com/,Real estate agency,Real estate agency,Real estate agency,+1 305-431-2258,"1761 W Flagler St, Miami, FL 33135",Little Havana,1761 W Flagler St,...,https://www.google.com/maps/place/Fausto+Comme...,ChIJp3fIEOq22YgRe6TSMoqqqbQ,13018100000000000000,24,4.2,Closed,Closed,9AM-6:30PM,0.622811,1.552958
1087,0x88d9b7d29524b2d5:0x4d2939c65d9c46ac,CLS Realty,http://miamiproperties.info/,"Real estate agency, Property investment compan...",Real estate agency,Real estate agency,+1 305-680-9696,"340 W Flagler St Suite 511, Miami, FL 33130",Downtown Miami,340 W Flagler St Suite 511,...,https://www.google.com/maps/place/CLS+Realty/@...,ChIJ1bIkldK32YgRrEacXcY5KU0,5560040000000000000,4,5.0,8AM-6PM,Closed,8AM-8PM,0.692516,1.592359
2612,0x88d9b69091f53b5b:0xaa4ba1d3f6458ac1,MV Real Estate Holdings LLC,None,Real estate agency,Real estate agency,Real estate agency,+1 305-233-1539,"123 SW North River Dr, Miami, FL 33130",Downtown Miami,123 SW North River Dr,...,https://www.google.com/maps/place/MV+Real+Esta...,ChIJWzv1kZC22YgRwYpF9tOhS6o,12271100000000000000,6,3.0,None,None,None,0.384433,1.631765
304,0x88d9b48676b1fc77:0x19c9393ae2245614,Jerome Morlot PA,https://www.morlotgroup.com/,"Real estate agent, Real estate consultant",Real estate agent,Real estate agent,+1 786-564-7790,"961 SW 7th St Apt 302, Miami, FL 33130",Little Havana,961 SW 7th St Apt 302,...,https://www.google.com/maps/place/Jerome+Morlo...,ChIJd_yxdoa02YgRFFYk4jo5yRk,1858080000000000000,16,5.0,9AM-5PM,9AM-3PM,9AM-8PM,0.746063,1.695554
1709,0x88d9b7fd1ab3e2a9:0xce51a35d568e721f,"Best of Luxury Realty | Brickell/Miami, FL",https://www.bestofluxuryrealty.com/Miami-Real-...,Real estate agency,Real estate agency,Real estate agency,+1 305-407-1657,"700 NW 1st Ave Ste 300, Miami, FL 33136",Overtown,700 NW 1st Ave Ste 300,...,https://www.google.com/maps/place/Best+of+Luxu...,ChIJqeKzGv232YgRH3KOVl2jUc4,14866800000000000000,1,5.0,Open 24 hours,Open 24 hours,Open 24 hours,0.655658,1.698914
2583,0x88d9b4288e30fa8b:0xe439371ad1297aab,Real Estate Investments by Borrero Corp.,https://ww